# Analysis of RVA Restaurants- Part 1: Scraping Restaurant Data From Yelp<br>
The goal of Part 1 is to scrape data on Richmond's top 50 restaurants. The data set is the result of searching for "Restaurants" in location: "Richmond, VA." We will then put this data into a .csv file via a pandas data frame. <br>
Columns: Name, Price, Rating, Category, # of ratings.<br><br>
The libraries we will be using in this part are:
* requests
* BeautifulSoup
* pandas

In [1]:
import requests
import bs4

In [2]:
base_url = 'https://www.yelp.com/search?cflt=restaurants&find_loc=Richmond%2C+VA&start={}'
page_code = [0,10,20]

In [111]:
url = 'https://www.yelp.com/search?cflt=restaurants&find_loc=Richmond%2C+VA'
res = requests.get(url).text

In [112]:
soup = bs4.BeautifulSoup(res,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;">
 <head>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/no-js/,"js");
  </script>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="en-US" http-equiv="Content-Language"/>
  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <meta content="" name="description"/>
  <link content="#c41200" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/97ea654d5227/assets/img/logos/yelp_burst.svg" rel="mask-icon" sizes="any"/>
  <link href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/118ff475a341/assets/img/logos/favicon.ico" rel="shortcut icon"/>
  <script>
   window.ga=window.ga||function(){(ga.q=ga.q||[]).push(arguments)};ga.l=+new Date;window.ygaPageStartTime=new Date().getTime(

In [120]:
t = soup.find_all('span', class_='lemon--span__09f24__3997G text__09f24__2tZKC text-color--black-regular__09f24__1QxyO text-align--left__09f24__3Drs0 text-weight--bold__09f24__WGVdT text-size--inherit__09f24__2rwpp')
for i in range (0,11):
    print(t[i].text)

NuVegan Cafe
1. Toast
2. Lucky AF
3. Secret Sandwich Society
4. Stella’s
5. Southbound
6. Hot Chick: A Fried Chicken Joint
7. Laura Lee’s
8. Lunch Or Supper
9. The Roosevelt
10. &pizza - Willow Lawn


### Finding the  parent 'div' element in each restaurant's "information box"

In [435]:
url = base_url.format(0)
res = requests.get(url).text
#print(url)

# Soup of pages 1 to 3
soup = bs4.BeautifulSoup(res, 'lxml')

# Looking for umbrella div/span element
container = soup.find_all('div', class_='lemon--div__09f24__1mboc mainAttributes__09f24__26-vh arrange-unit__09f24__1gZC1 arrange-unit-fill__09f24__O6JFU border-color--default__09f24__R1nRO')

#### Searching through this new container variable for appropriate elements:
Example: This is restaurant #9's container

In [436]:
print(container[9].prettify())

<div class="lemon--div__09f24__1mboc mainAttributes__09f24__26-vh arrange-unit__09f24__1gZC1 arrange-unit-fill__09f24__O6JFU border-color--default__09f24__R1nRO">
 <div class="lemon--div__09f24__1mboc border-color--default__09f24__R1nRO">
  <div class="lemon--div__09f24__1mboc border-color--default__09f24__R1nRO">
   <div class="lemon--div__09f24__1mboc businessName__09f24__3Wql2 display--inline-block__09f24__FsgS4 border-color--default__09f24__R1nRO">
    <div class="lemon--div__09f24__1mboc border-color--default__09f24__R1nRO">
     <h4 class="lemon--h4__09f24__1yd__ heading--h4__09f24__2ijYq alternate__09f24__39r7c">
      <span class="lemon--span__09f24__3997G text__09f24__2tZKC text-color--black-regular__09f24__1QxyO text-align--left__09f24__3Drs0 text-weight--bold__09f24__WGVdT text-size--inherit__09f24__2rwpp">
       9
       <!-- -->
       .
       <a class="lemon--a__09f24__IEZFH link__09f24__1kwXV link-color--inherit__09f24__3PYlA link-size--inherit__09f24__2Uj95" href="/bi

#### Getting Restaurant Names:
Example: Restaurant #9

In [499]:
name_span = container[9].find('span', class_='lemon--span__09f24__3997G text__09f24__2tZKC text-color--black-regular__09f24__1QxyO text-align--left__09f24__3Drs0 text-weight--bold__09f24__WGVdT text-size--inherit__09f24__2rwpp')

# MUST use print or else weird markdown becomes available
print(name_span.text)

9. Mama J’s Kitchen


#### Getting Ratings:

In [500]:
rating_span = container[9].find('span', class_='lemon--span__09f24__3997G display--inline__09f24__3iACj border-color--default__09f24__R1nRO')
rating_span.find('div')['aria-label']

'4.5 star rating'

#### Getting Review Counts:

In [501]:
review_cnt_span = container[9].find('span', class_='lemon--span__09f24__3997G text__09f24__2tZKC reviewCount__09f24__EUXPN text-color--black-extra-light__09f24__38DtK text-align--left__09f24__3Drs0')
review_cnt_span.text

'1052'

#### Getting Price Range (if available):

In [502]:
price_span = container[9].find('span', class_='lemon--span__09f24__3997G text__09f24__2tZKC priceRange__09f24__2O6le text-color--black-extra-light__09f24__38DtK text-align--left__09f24__3Drs0 text-bullet--after__09f24__1MWoX')
price_span.text

'$$'

*However, not all restaurants have a price range available. If the variable is "None" it won't print. Replace it with "N/A"

In [26]:
for i in range(0, len(container)):
    price_span = container[i].find('span', class_='lemon--span__09f24__3997G text__09f24__2tZKC priceRange__09f24__2O6le text-color--black-extra-light__09f24__38DtK text-align--left__09f24__3Drs0 text-bullet--after__09f24__1MWoX')
    if price_span == None:
        print('N/A')
    else:
        print(price_span.text)

$
$$$
$$
$$
$$
$$
$$
$
$$
$$
$$


#### Grabbing Restaurant Categories:

In [471]:
tag_span = container[9].find_all('span',class_='lemon--span__09f24__3997G text__09f24__2tZKC text-color--black-extra-light__09f24__38DtK text-align--left__09f24__3Drs0')
tag_span

[<span class="lemon--span__09f24__3997G text__09f24__2tZKC text-color--black-extra-light__09f24__38DtK text-align--left__09f24__3Drs0"><a class="lemon--a__09f24__IEZFH link__09f24__1kwXV link-color--inherit__09f24__3PYlA link-size--default__09f24__3xWLF" href="/search?cflt=southern&amp;find_loc=Richmond%2C+VA" name="" rel="" role="link" target="">Southern</a>, </span>,
 <span class="lemon--span__09f24__3997G text__09f24__2tZKC text-color--black-extra-light__09f24__38DtK text-align--left__09f24__3Drs0"><a class="lemon--a__09f24__IEZFH link__09f24__1kwXV link-color--inherit__09f24__3PYlA link-size--default__09f24__3xWLF" href="/search?cflt=newamerican&amp;find_loc=Richmond%2C+VA" name="" rel="" role="link" target="">American (New)</a></span>]

In [477]:
tags = []
for tag in tag_span:
    tags.append(tag.text)
print(tags)

['Southern, ', 'American (New)']


### Test Loop through Page 1

In [5]:
url = base_url.format(0)
res = requests.get(url).text
#print(url)

# Add lists

# Soup of pages 1 to 3
soup = bs4.BeautifulSoup(res, 'lxml')

# Looking for umbrella div/span element
container = soup.find_all('div', class_='lemon--div__09f24__1mboc mainAttributes__09f24__26-vh arrange-unit__09f24__1gZC1 arrange-unit-fill__09f24__O6JFU border-color--default__09f24__R1nRO')

print('Remove NuVegan with if/else after testing\n')
for box in range(0,len(container)):
    name_span = container[box].find('span', class_='lemon--span__09f24__3997G text__09f24__2tZKC text-color--black-regular__09f24__1QxyO text-align--left__09f24__3Drs0 text-weight--bold__09f24__WGVdT text-size--inherit__09f24__2rwpp')
    
    #if '.' not in name_span.text:
        #pass
    
    #else:
    print(name_span.text)

    rating_span = container[box].find('span', class_='lemon--span__09f24__3997G display--inline__09f24__3iACj border-color--default__09f24__R1nRO')
    print(rating_span.find('div')['aria-label'])
    
    price_span = container[box].find('span', class_='lemon--span__09f24__3997G text__09f24__2tZKC priceRange__09f24__2O6le text-color--black-extra-light__09f24__38DtK text-align--left__09f24__3Drs0 text-bullet--after__09f24__1MWoX')
    if price_span == None:
        print('N/A')
    else:
        print(price_span.text)
    
    tag_span = container[box].find_all('span',class_='lemon--span__09f24__3997G text__09f24__2tZKC text-color--black-extra-light__09f24__38DtK text-align--left__09f24__3Drs0')
    tags = []
    for tag in tag_span:
        # Clean up comma
        tags.append(tag.text)
    print(tags)

    print('----')

Remove NuVegan with if/else after testing

NuVegan Cafe
3.5 star rating
$
['Cafes, ', 'Vegan']
----
1. Toast
4 star rating
$$
['Southern, ', 'Gastropubs']
----
2. Lucky AF
5 star rating
N/A
['Asian Fusion, ', 'Sushi Bars, ', 'Pan Asian']
----
3. Secret Sandwich Society
4.5 star rating
$$
['Burgers, ', 'Sandwiches, ', 'Cocktail Bars']
----
4. Stella’s
4.5 star rating
$$
['Mediterranean, ', 'Greek, ', 'Cocktail Bars']
----
5. Lunch Or Supper
4.5 star rating
$$
['American (New), ', 'Breakfast & Brunch, ', 'Southern']
----
6. Laura Lee’s
4.5 star rating
$$
['American (New)']
----
7. Hot Chick: A Fried Chicken Joint
4 star rating
$$
['Chicken Shop']
----
8. ZZQ Texas Craft Barbeque
4.5 star rating
$$
['Barbeque']
----
9. Mama J’s Kitchen
4.5 star rating
$$
['Soul Food, ', 'Bars, ', 'Sandwiches']
----
10. &pizza - Willow Lawn
5 star rating
N/A
['Pizza']
----


## Loop through Pages 1-5 to get Top 50:

In [9]:
base_url = 'https://www.yelp.com/search?cflt=restaurants&find_loc=Richmond%2C+VA&start={}'
page_code = [0,10,20,30,40]

#Lists
name = []
rating = []
review_count = []
price = []
categories = []

for page in page_code:
    url = base_url.format(page)
    res = requests.get(url).text
    #print(url)

    # Add lists

    # Soup of pages 1 to 3
    soup = bs4.BeautifulSoup(res, 'lxml')

    # Looking for umbrella div/span element
    container = soup.find_all('div', class_='lemon--div__09f24__1mboc mainAttributes__09f24__26-vh arrange-unit__09f24__1gZC1 arrange-unit-fill__09f24__O6JFU border-color--default__09f24__R1nRO')


    for box in range(0,len(container)):
        name_span = container[box].find('span', class_='lemon--span__09f24__3997G text__09f24__2tZKC text-color--black-regular__09f24__1QxyO text-align--left__09f24__3Drs0 text-weight--bold__09f24__WGVdT text-size--inherit__09f24__2rwpp')

        if '.' not in name_span.text:
            pass

        else:
            name.append(name_span.text)
            print(name_span.text)

            rating_span = container[box].find('span', class_='lemon--span__09f24__3997G display--inline__09f24__3iACj border-color--default__09f24__R1nRO')
            rating.append(rating_span.find('div')['aria-label'])
            print(rating_span.find('div')['aria-label'])
            
            review_cnt_span = container[box].find('span', class_='lemon--span__09f24__3997G text__09f24__2tZKC reviewCount__09f24__EUXPN text-color--black-extra-light__09f24__38DtK text-align--left__09f24__3Drs0')
            print(review_cnt_span.text)
            review_count.append(review_cnt_span.text)

            price_span = container[box].find('span', class_='lemon--span__09f24__3997G text__09f24__2tZKC priceRange__09f24__2O6le text-color--black-extra-light__09f24__38DtK text-align--left__09f24__3Drs0 text-bullet--after__09f24__1MWoX')
            if price_span == None:
                dollar_signs = ('N/A')
                print(dollar_signs)
                price.append(dollar_signs)
                
            else:
                price.append(price_span.text)
                print(price_span.text)

            tag_span = container[box].find_all('span',class_='lemon--span__09f24__3997G text__09f24__2tZKC text-color--black-extra-light__09f24__38DtK text-align--left__09f24__3Drs0')
            tags = []
            for tag in tag_span:
                # Clean up comma
                tags.append(tag.text)
            print(tags)
            categories.append(tags)

            print('----')

1. Toast
4 star rating
619
$$
['Southern, ', 'Gastropubs']
----
2. Lucky AF
5 star rating
9
N/A
['Asian Fusion, ', 'Sushi Bars, ', 'Pan Asian']
----
3. Stella’s
4.5 star rating
715
$$
['Mediterranean, ', 'Greek, ', 'Cocktail Bars']
----
4. Secret Sandwich Society
4.5 star rating
936
$$
['Burgers, ', 'Sandwiches, ', 'Cocktail Bars']
----
5. Laura Lee’s
4.5 star rating
260
$$
['American (New)']
----
6. Lunch Or Supper
4.5 star rating
1633
$$
['American (New), ', 'Breakfast & Brunch, ', 'Southern']
----
7. Southbound
4 star rating
315
$$
['Southern, ', 'American (New)']
----
8. Mama J’s Kitchen
4.5 star rating
1052
$$
['Soul Food, ', 'Bars, ', 'Sandwiches']
----
9. Carmela’s
5 star rating
91
$$
['Pizza, ', 'Italian']
----
10. Latitude Seafood - Richmond
3.5 star rating
134
$$
['Seafood']
----
11. Mama J’s Kitchen
4.5 star rating
1052
$$
['Soul Food, ', 'Bars, ', 'Sandwiches']
----
12. Carmela’s
5 star rating
91
$$
['Pizza, ', 'Italian']
----
13. The Roosevelt
4 star rating
481
$$
['Southe

#### Let's check our lists, then add them to a data frame for further cleaning when complete!

In [10]:
name

['1.\xa0Toast',
 '2.\xa0Lucky AF',
 '3.\xa0Stella’s',
 '4.\xa0Secret Sandwich Society',
 '5.\xa0Laura Lee’s',
 '6.\xa0Lunch Or Supper',
 '7.\xa0Southbound',
 '8.\xa0Mama J’s Kitchen',
 '9.\xa0Carmela’s',
 '10.\xa0Latitude Seafood - Richmond',
 '11.\xa0Mama J’s Kitchen',
 '12.\xa0Carmela’s',
 '13.\xa0The Roosevelt',
 '14.\xa0Latitude Seafood - Richmond',
 '15.\xa0&pizza - Willow Lawn',
 '16.\xa0Perch',
 '17.\xa0Little Nickel',
 '18.\xa0Heritage - Temp. CLOSED',
 '19.\xa0Sabai',
 '20.\xa0Addis Ethiopian Restaurant',
 '21.\xa0Jack Brown’s Beer and Burger Joint',
 '22.\xa0The Broken Tulip',
 '23.\xa0Croaker’s Spot Restaurant',
 '24.\xa0Foo Dog',
 '25.\xa0Tipsy Crab',
 '26.\xa0Natalie’s Taste of Lebanon',
 '27.\xa0Beauvine Burger Concept',
 '28.\xa0Kuba Kuba',
 '29.\xa0The Continental Westhampton',
 '30.\xa0&pizza - Willow Lawn',
 '31.\xa0Abuelita’s',
 '32.\xa0Greek On Cary',
 '33.\xa0Sabai',
 '34.\xa0Natalie’s Taste of Lebanon',
 '35.\xa0Heritage - Temp. CLOSED',
 '36.\xa0Addis Ethiopian R

In [11]:
rating

['4 star rating',
 '5 star rating',
 '4.5 star rating',
 '4.5 star rating',
 '4.5 star rating',
 '4.5 star rating',
 '4 star rating',
 '4.5 star rating',
 '5 star rating',
 '3.5 star rating',
 '4.5 star rating',
 '5 star rating',
 '4 star rating',
 '3.5 star rating',
 '5 star rating',
 '4.5 star rating',
 '4.5 star rating',
 '4.5 star rating',
 '4 star rating',
 '4 star rating',
 '4 star rating',
 '5 star rating',
 '4 star rating',
 '4 star rating',
 '4 star rating',
 '4.5 star rating',
 '4.5 star rating',
 '4 star rating',
 '4 star rating',
 '5 star rating',
 '5 star rating',
 '4 star rating',
 '4 star rating',
 '4.5 star rating',
 '4.5 star rating',
 '4 star rating',
 '4 star rating',
 '4 star rating',
 '4 star rating',
 '4.5 star rating',
 '4 star rating',
 '4.5 star rating',
 '4.5 star rating',
 '4.5 star rating',
 '4 star rating',
 '4 star rating',
 '4.5 star rating',
 '4 star rating',
 '4 star rating',
 '4 star rating']

In [14]:
review_count

['619',
 '9',
 '715',
 '936',
 '260',
 '1633',
 '315',
 '1052',
 '91',
 '134',
 '1052',
 '91',
 '481',
 '134',
 '5',
 '145',
 '233',
 '606',
 '442',
 '258',
 '248',
 '41',
 '639',
 '446',
 '125',
 '196',
 '370',
 '853',
 '377',
 '5',
 '160',
 '486',
 '442',
 '196',
 '606',
 '258',
 '384',
 '473',
 '279',
 '236',
 '279',
 '187',
 '97',
 '349',
 '89',
 '233',
 '265',
 '125',
 '384',
 '193']

In [12]:
price

['$$',
 'N/A',
 '$$',
 '$$',
 '$$',
 '$$',
 '$$',
 '$$',
 '$$',
 '$$',
 '$$',
 '$$',
 '$$',
 '$$',
 'N/A',
 'N/A',
 '$$',
 '$$',
 '$$',
 '$$',
 '$$',
 'N/A',
 '$$',
 '$$',
 '$$',
 '$$',
 '$$',
 '$$',
 '$$',
 'N/A',
 '$',
 '$$',
 '$$',
 '$$',
 '$$',
 '$$',
 '$$',
 '$$',
 '$$$',
 '$$$',
 '$$$',
 '$$',
 '$$',
 '$$',
 '$$',
 '$$',
 '$',
 '$$',
 '$$',
 '$$']

In [13]:
categories

[['Southern, ', 'Gastropubs'],
 ['Asian Fusion, ', 'Sushi Bars, ', 'Pan Asian'],
 ['Mediterranean, ', 'Greek, ', 'Cocktail Bars'],
 ['Burgers, ', 'Sandwiches, ', 'Cocktail Bars'],
 ['American (New)'],
 ['American (New), ', 'Breakfast & Brunch, ', 'Southern'],
 ['Southern, ', 'American (New)'],
 ['Soul Food, ', 'Bars, ', 'Sandwiches'],
 ['Pizza, ', 'Italian'],
 ['Seafood'],
 ['Soul Food, ', 'Bars, ', 'Sandwiches'],
 ['Pizza, ', 'Italian'],
 ['Southern, ', 'Cocktail Bars'],
 ['Seafood'],
 ['Pizza'],
 ['American (New)'],
 ['American (Traditional), ', 'Mediterranean, ', 'Tiki Bars'],
 ['American (New), ', 'Bars'],
 ['Bars, ', 'Thai'],
 ['Ethiopian'],
 ['Burgers, ', 'Beer Bar'],
 ['American (New)'],
 ['Soul Food, ', 'Seafood, ', 'Southern'],
 ['Cocktail Bars, ', 'Asian Fusion'],
 ['Seafood'],
 ['Lebanese, ', 'Mediterranean, ', 'Venues & Event Spaces'],
 ['Burgers, ', 'American (Traditional)'],
 ['Cuban'],
 ['American (New)'],
 ['Pizza'],
 ['Mexican'],
 ['Greek, ', 'Mediterranean'],
 ['Bars,

___

### Create DataFrame from lists then save to .csv in directory
Import numpy and pandas packages

In [15]:
import numpy as np
import pandas as pd

In [35]:
df = pd.DataFrame({'name': name, 'rating': rating, 'price': price, 'review_count': review_count, 'categories':categories})

In [36]:
df

,name,rating,price,review_count,categories
0,1. Toast,4 star rating,$$,619,"[Southern, , Gastropubs]"
1,2. Lucky AF,5 star rating,N/A,9,"[Asian Fusion, , Sushi Bars, , Pan Asian]"
2,3. Stella’s,4.5 star rating,$$,715,"[Mediterranean, , Greek, , Cocktail Bars]"
3,4. Secret Sandwich Society,4.5 star rating,$$,936,"[Burgers, , Sandwiches, , Cocktail Bars]"
4,5. Laura Lee’s,4.5 star rating,$$,260,[American (New)]
5,6. Lunch Or Supper,4.5 star rating,$$,1633,"[American (New), , Breakfast & Brunch, , South..."
6,7. Southbound,4 star rating,$$,315,"[Southern, , American (New)]"
7,8. Mama J’s Kitchen,4.5 star rating,$$,1052,"[Soul Food, , Bars, , Sandwiches]"
8,9. Carmela’s,5 star rating,$$,91,"[Pizza, , Italian]"
9,10. Latitude Seafood - Richmond,3.5 star rating,$$,134,[Seafood]


#### Our dataframe is created! Last step is to save into a CSV file to transfer to another notebook for Part 2.

In [37]:
df.to_csv('/Users/terencerustia/Documents/Data Science Projects/Yelp Data Analysis/rva_restaurants_scraped.csv', index=False)
# index = False makes sure you don't save the index number into the csv file.

#### All done. We'll move on to Part 2 in another notebook, where we will clean the data!